# Story Generation Exercise Solutions

## Solution 1: CSV Renderer

In [ ]:
from odibi.story.metadata import PipelineStoryMetadata, NodeExecutionMetadata
import csv
from io import StringIO
from pathlib import Path

class CSVStoryRenderer:
    """Renders pipeline stories as CSV."""
    
    def render(self, metadata: PipelineStoryMetadata) -> str:
        """Render story as CSV string."""
        output = StringIO()
        
        fieldnames = [
            'node_name', 
            'operation', 
            'status', 
            'duration', 
            'rows_in', 
            'rows_out',
            'rows_change',
            'rows_change_pct'
        ]
        
        writer = csv.DictWriter(output, fieldnames=fieldnames)
        writer.writeheader()
        
        # Write node rows
        for node in metadata.nodes:
            writer.writerow({
                'node_name': node.node_name,
                'operation': node.operation,
                'status': node.status,
                'duration': node.duration,
                'rows_in': node.rows_in or '',
                'rows_out': node.rows_out or '',
                'rows_change': node.rows_change or '',
                'rows_change_pct': f"{node.rows_change_pct:.2f}" if node.rows_change_pct is not None else ''
            })
        
        # Add summary row
        writer.writerow({
            'node_name': 'PIPELINE_SUMMARY',
            'operation': metadata.pipeline_name,
            'status': 'success' if metadata.failed_nodes == 0 else 'failed',
            'duration': metadata.duration,
            'rows_in': '',
            'rows_out': metadata.get_total_rows_processed(),
            'rows_change': '',
            'rows_change_pct': f"{metadata.get_success_rate():.1f}"
        })
        
        return output.getvalue()
    
    def render_to_file(self, metadata: PipelineStoryMetadata, output_path: str) -> str:
        """Render story and save to CSV file."""
        csv_content = self.render(metadata)
        
        output_file = Path(output_path)
        output_file.parent.mkdir(parents=True, exist_ok=True)
        
        with open(output_file, 'w', encoding='utf-8', newline='') as f:
            f.write(csv_content)
        
        return str(output_file)

# Test implementation
test_metadata = PipelineStoryMetadata(
    pipeline_name="test_pipeline",
    duration=10.5
)

node1 = NodeExecutionMetadata(
    node_name="extract",
    operation="read_csv",
    status="success",
    duration=3.5,
    rows_in=0,
    rows_out=1000
)
node1.calculate_row_change()
test_metadata.add_node(node1)

node2 = NodeExecutionMetadata(
    node_name="transform",
    operation="filter",
    status="success",
    duration=2.0,
    rows_in=1000,
    rows_out=850
)
node2.calculate_row_change()
test_metadata.add_node(node2)

renderer = CSVStoryRenderer()
csv_output = renderer.render(test_metadata)
print(csv_output)

## Solution 2: Enhanced Metadata with Memory Tracking

In [ ]:
from dataclasses import dataclass, field
from typing import Optional, List, Dict, Any

@dataclass
class EnhancedNodeExecutionMetadata:
    """Extended metadata with memory tracking."""
    
    node_name: str
    operation: str
    status: str
    duration: float
    
    # Data metrics
    rows_in: Optional[int] = None
    rows_out: Optional[int] = None
    rows_change: Optional[int] = None
    rows_change_pct: Optional[float] = None
    
    # Memory metrics
    memory_mb: Optional[float] = None
    peak_memory_mb: Optional[float] = None
    memory_efficiency: Optional[float] = None  # rows per MB
    
    # Schema tracking
    schema_in: Optional[List[str]] = None
    schema_out: Optional[List[str]] = None
    
    def calculate_row_change(self):
        """Calculate row count change metrics."""
        if self.rows_in is not None and self.rows_out is not None:
            self.rows_change = self.rows_out - self.rows_in
            if self.rows_in > 0:
                self.rows_change_pct = (self.rows_change / self.rows_in) * 100
            else:
                self.rows_change_pct = 0.0 if self.rows_out == 0 else 100.0
    
    def calculate_memory_efficiency(self) -> Optional[float]:
        """Calculate rows processed per MB of memory."""
        if self.rows_out is not None and self.memory_mb and self.memory_mb > 0:
            self.memory_efficiency = self.rows_out / self.memory_mb
            return self.memory_efficiency
        return None
    
    def to_dict(self) -> Dict[str, Any]:
        """Convert to dictionary."""
        return {
            "node_name": self.node_name,
            "operation": self.operation,
            "status": self.status,
            "duration": self.duration,
            "rows_in": self.rows_in,
            "rows_out": self.rows_out,
            "rows_change": self.rows_change,
            "rows_change_pct": self.rows_change_pct,
            "memory_mb": self.memory_mb,
            "peak_memory_mb": self.peak_memory_mb,
            "memory_efficiency": self.memory_efficiency,
            "schema_in": self.schema_in,
            "schema_out": self.schema_out,
        }

# Test implementation
node = EnhancedNodeExecutionMetadata(
    node_name="memory_test",
    operation="transform",
    status="success",
    duration=5.0,
    rows_out=100000,
    memory_mb=256.5,
    peak_memory_mb=312.8
)

efficiency = node.calculate_memory_efficiency()
print(f"Memory Efficiency: {efficiency:.2f} rows/MB")
print(f"Peak Memory: {node.peak_memory_mb:.2f} MB")
print(f"\nMetadata Dict:")
import json
print(json.dumps(node.to_dict(), indent=2))

## Solution 3: Custom Organization Theme

In [ ]:
from odibi.story.themes import StoryTheme

def create_organization_theme():
    """Create a branded theme for Ingredion."""
    
    theme = StoryTheme(
        name="ingredion_corporate",
        
        # Color scheme - Ingredion branding
        primary_color="#006837",      # Ingredion green
        success_color="#2e7d32",      # Green for success
        error_color="#c62828",        # Red for errors
        warning_color="#f57c00",      # Orange for warnings
        bg_color="#fafafa",           # Light background
        text_color="#212121",         # Dark text
        border_color="#e0e0e0",       # Subtle borders
        code_bg="#f5f5f5",            # Code background
        
        # Typography
        font_family="'Open Sans', 'Helvetica Neue', Arial, sans-serif",
        heading_font="'Roboto', 'Open Sans', Arial, sans-serif",
        code_font="'Fira Code', Consolas, Monaco, monospace",
        font_size="15px",
        
        # Branding
        logo_url="https://www.ingredion.com/content/dam/ingredion/logos/ingredion-logo.png",
        company_name="Ingredion Incorporated",
        footer_text="© 2024 Ingredion - Confidential & Proprietary",
        
        # Layout
        max_width="1400px",
        sidebar=False,
        
        # Custom CSS
        custom_css="""
        /* Card styling with rounded corners */
        .node-card {
            border-radius: 8px;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
            transition: box-shadow 0.3s ease;
        }
        
        .node-card:hover {
            box-shadow: 0 4px 8px rgba(0,0,0,0.15);
        }
        
        /* Gradient header */
        .story-header {
            background: linear-gradient(135deg, #006837 0%, #008c4a 100%);
            color: white;
            padding: 24px;
            border-radius: 8px 8px 0 0;
            margin-bottom: 20px;
        }
        
        /* Status badges with rounded style */
        .status-badge {
            display: inline-block;
            padding: 4px 12px;
            border-radius: 12px;
            font-weight: 600;
            font-size: 13px;
        }
        
        .status-success {
            background-color: #e8f5e9;
            color: #2e7d32;
        }
        
        .status-error {
            background-color: #ffebee;
            color: #c62828;
        }
        
        /* Hover effects on interactive elements */
        .expandable-section {
            cursor: pointer;
            padding: 12px;
            border-left: 4px solid var(--primary-color);
            transition: background-color 0.2s ease;
        }
        
        .expandable-section:hover {
            background-color: rgba(0, 104, 55, 0.05);
        }
        
        /* Data tables */
        .data-table {
            border-radius: 4px;
            overflow: hidden;
        }
        
        .data-table th {
            background-color: #006837;
            color: white;
            padding: 12px;
        }
        
        .data-table td {
            padding: 10px;
            border-bottom: 1px solid #e0e0e0;
        }
        
        .data-table tr:hover {
            background-color: #f5f5f5;
        }
        """
    )
    
    return theme

# Test the theme
my_theme = create_organization_theme()
print("Ingredion Corporate Theme")
print("=" * 50)
print("\nCSS Variables:")
for var, value in my_theme.to_css_vars().items():
    print(f"  {var}: {value}")

print("\n\nComplete CSS:")
print(my_theme.to_css_string()[:500] + "\n...")

## Solution 4: Story Diff Implementation

In [ ]:
from typing import List, Tuple, Dict, Any
from odibi.story.metadata import PipelineStoryMetadata, NodeExecutionMetadata

class StoryDiff:
    """Compare two pipeline story executions."""
    
    def __init__(self, before: PipelineStoryMetadata, after: PipelineStoryMetadata):
        self.before = before
        self.after = after
    
    def compare_performance(self) -> Dict[str, Any]:
        """Compare overall performance metrics."""
        duration_diff = self.after.duration - self.before.duration
        duration_pct = (duration_diff / self.before.duration * 100) if self.before.duration > 0 else 0
        
        success_rate_diff = self.after.get_success_rate() - self.before.get_success_rate()
        
        return {
            'duration_before': self.before.duration,
            'duration_after': self.after.duration,
            'duration_diff': duration_diff,
            'duration_pct_change': duration_pct,
            'success_rate_before': self.before.get_success_rate(),
            'success_rate_after': self.after.get_success_rate(),
            'success_rate_diff': success_rate_diff
        }
    
    def compare_nodes(self) -> Dict[str, List[str]]:
        """Identify node changes."""
        before_nodes = {node.node_name for node in self.before.nodes}
        after_nodes = {node.node_name for node in self.after.nodes}
        
        added = list(after_nodes - before_nodes)
        removed = list(before_nodes - after_nodes)
        common = list(before_nodes & after_nodes)
        
        # Find nodes with status changes
        status_changes = []
        before_dict = {n.node_name: n for n in self.before.nodes}
        after_dict = {n.node_name: n for n in self.after.nodes}
        
        for node_name in common:
            if before_dict[node_name].status != after_dict[node_name].status:
                status_changes.append(
                    f"{node_name}: {before_dict[node_name].status} -> {after_dict[node_name].status}"
                )
        
        return {
            'added': added,
            'removed': removed,
            'common': common,
            'status_changes': status_changes
        }
    
    def compare_data_volume(self) -> Dict[str, Any]:
        """Compare data processing volumes."""
        before_total = self.before.get_total_rows_processed()
        after_total = self.after.get_total_rows_processed()
        
        diff = after_total - before_total
        pct = (diff / before_total * 100) if before_total > 0 else 0
        
        # Compare node-level data volumes
        node_changes = []
        before_dict = {n.node_name: n for n in self.before.nodes}
        after_dict = {n.node_name: n for n in self.after.nodes}
        
        for node_name in set(before_dict.keys()) & set(after_dict.keys()):
            before_rows = before_dict[node_name].rows_out or 0
            after_rows = after_dict[node_name].rows_out or 0
            
            if before_rows != after_rows:
                node_diff = after_rows - before_rows
                node_pct = (node_diff / before_rows * 100) if before_rows > 0 else 0
                node_changes.append({
                    'node_name': node_name,
                    'before': before_rows,
                    'after': after_rows,
                    'diff': node_diff,
                    'pct': node_pct
                })
        
        return {
            'total_before': before_total,
            'total_after': after_total,
            'total_diff': diff,
            'total_pct_change': pct,
            'node_changes': node_changes
        }
    
    def generate_report(self) -> str:
        """Generate markdown diff report."""
        lines = []
        
        # Header
        lines.append(f"# Pipeline Comparison Report")
        lines.append(f"")
        lines.append(f"**Pipeline:** {self.before.pipeline_name}")
        lines.append(f"**Before:** {self.before.started_at}")
        lines.append(f"**After:** {self.after.started_at}")
        lines.append(f"")
        lines.append("---")
        lines.append(f"")
        
        # Performance comparison
        perf = self.compare_performance()
        lines.append("## Performance Comparison")
        lines.append(f"")
        
        duration_icon = "🟢" if perf['duration_diff'] < 0 else "🔴" if perf['duration_diff'] > 0 else "⚪"
        lines.append(f"{duration_icon} **Duration:**")
        lines.append(f"  - Before: {perf['duration_before']:.2f}s")
        lines.append(f"  - After: {perf['duration_after']:.2f}s")
        lines.append(f"  - Change: {perf['duration_diff']:+.2f}s ({perf['duration_pct_change']:+.1f}%)")
        lines.append(f"")
        
        success_icon = "🟢" if perf['success_rate_diff'] > 0 else "🔴" if perf['success_rate_diff'] < 0 else "⚪"
        lines.append(f"{success_icon} **Success Rate:**")
        lines.append(f"  - Before: {perf['success_rate_before']:.1f}%")
        lines.append(f"  - After: {perf['success_rate_after']:.1f}%")
        lines.append(f"  - Change: {perf['success_rate_diff']:+.1f}%")
        lines.append(f"")
        
        # Node changes
        nodes = self.compare_nodes()
        lines.append("## Node Changes")
        lines.append(f"")
        
        if nodes['added']:
            lines.append(f"➕ **Added Nodes:** {', '.join(nodes['added'])}")
        if nodes['removed']:
            lines.append(f"➖ **Removed Nodes:** {', '.join(nodes['removed'])}")
        if nodes['status_changes']:
            lines.append(f"⚠️ **Status Changes:**")
            for change in nodes['status_changes']:
                lines.append(f"  - {change}")
        if not (nodes['added'] or nodes['removed'] or nodes['status_changes']):
            lines.append("✅ No node changes")
        lines.append(f"")
        
        # Data volume changes
        volume = self.compare_data_volume()
        lines.append("## Data Volume Changes")
        lines.append(f"")
        lines.append(f"**Total Rows Processed:**")
        lines.append(f"  - Before: {volume['total_before']:,}")
        lines.append(f"  - After: {volume['total_after']:,}")
        lines.append(f"  - Change: {volume['total_diff']:+,} ({volume['total_pct_change']:+.1f}%)")
        lines.append(f"")
        
        if volume['node_changes']:
            lines.append("**Node-Level Changes:**")
            lines.append(f"")
            lines.append("| Node | Before | After | Change | % Change |")
            lines.append("|------|--------|-------|--------|----------|")
            for nc in volume['node_changes']:
                lines.append(
                    f"| {nc['node_name']} | {nc['before']:,} | {nc['after']:,} | "
                    f"{nc['diff']:+,} | {nc['pct']:+.1f}% |"
                )
        
        return "\n".join(lines)

# Test the implementation
run1 = PipelineStoryMetadata(
    pipeline_name="test",
    duration=10.0,
    started_at="2024-01-15T10:00:00"
)
n1 = NodeExecutionMetadata(
    node_name="extract",
    operation="read",
    status="success",
    duration=3.0,
    rows_out=1000
)
run1.add_node(n1)

n2 = NodeExecutionMetadata(
    node_name="transform",
    operation="filter",
    status="success",
    duration=7.0,
    rows_in=1000,
    rows_out=900
)
run1.add_node(n2)

run2 = PipelineStoryMetadata(
    pipeline_name="test",
    duration=8.0,
    started_at="2024-01-15T11:00:00"
)
n3 = NodeExecutionMetadata(
    node_name="extract",
    operation="read",
    status="success",
    duration=2.5,
    rows_out=1000
)
run2.add_node(n3)

n4 = NodeExecutionMetadata(
    node_name="transform",
    operation="filter",
    status="success",
    duration=5.5,
    rows_in=1000,
    rows_out=900
)
run2.add_node(n4)

diff = StoryDiff(run1, run2)
report = diff.generate_report()
print(report)

## Solution 5: Story Analytics Dashboard

In [ ]:
from pathlib import Path
from collections import Counter, defaultdict
from typing import List, Tuple, Dict, Any
import json

class StoryAnalyzer:
    """Analyze multiple pipeline story executions."""
    
    def __init__(self):
        self.stories: List[Dict] = []
    
    def load_from_directory(self, directory: str):
        """Load all JSON story files from directory."""
        directory_path = Path(directory)
        for json_file in directory_path.glob('*.json'):
            try:
                with open(json_file, 'r') as f:
                    story = json.load(f)
                    self.stories.append(story)
            except Exception as e:
                print(f"Warning: Could not load {json_file}: {e}")
    
    def add_story(self, story_dict: Dict):
        """Add a story to the analysis."""
        self.stories.append(story_dict)
    
    def get_average_duration(self) -> float:
        """Calculate average pipeline duration."""
        if not self.stories:
            return 0.0
        
        total = sum(story.get('duration', 0) for story in self.stories)
        return total / len(self.stories)
    
    def get_failure_hotspots(self) -> List[Tuple[str, int]]:
        """Find nodes that fail most frequently."""
        failure_counts = Counter()
        
        for story in self.stories:
            for node in story.get('nodes', []):
                if node.get('status') == 'failed':
                    failure_counts[node['node_name']] += 1
        
        return failure_counts.most_common()
    
    def get_slowest_nodes(self, top_n: int = 5) -> List[Tuple[str, float]]:
        """Find slowest nodes across all runs."""
        node_durations = defaultdict(list)
        
        # Collect all durations for each node
        for story in self.stories:
            for node in story.get('nodes', []):
                node_name = node['node_name']
                duration = node.get('duration', 0)
                node_durations[node_name].append(duration)
        
        # Calculate average duration for each node
        avg_durations = {
            name: sum(durations) / len(durations)
            for name, durations in node_durations.items()
        }
        
        # Sort and return top N
        sorted_nodes = sorted(avg_durations.items(), key=lambda x: x[1], reverse=True)
        return sorted_nodes[:top_n]
    
    def get_total_data_processed(self) -> int:
        """Calculate total rows processed across all runs."""
        total = 0
        for story in self.stories:
            total += story.get('total_rows_processed', 0)
        return total
    
    def get_success_rate_trend(self) -> List[float]:
        """Get success rate for each run in chronological order."""
        # Sort by started_at
        sorted_stories = sorted(
            self.stories,
            key=lambda s: s.get('started_at', '')
        )
        
        return [story.get('success_rate', 0) for story in sorted_stories]
    
    def generate_dashboard(self) -> str:
        """Generate markdown analytics dashboard."""
        lines = []
        
        # Header
        lines.append("# Pipeline Analytics Dashboard")
        lines.append("")
        lines.append(f"**Total Runs Analyzed:** {len(self.stories)}")
        lines.append("")
        lines.append("---")
        lines.append("")
        
        # Overall metrics
        lines.append("## Overall Metrics")
        lines.append("")
        
        avg_duration = self.get_average_duration()
        total_data = self.get_total_data_processed()
        
        lines.append(f"- ⏱️ **Average Duration:** {avg_duration:.2f}s")
        lines.append(f"- 📊 **Total Data Processed:** {total_data:,} rows")
        
        if self.stories:
            avg_success = sum(s.get('success_rate', 0) for s in self.stories) / len(self.stories)
            lines.append(f"- ✅ **Average Success Rate:** {avg_success:.1f}%")
        
        lines.append("")
        
        # Failure hotspots
        lines.append("## Failure Hotspots")
        lines.append("")
        
        failures = self.get_failure_hotspots()
        if failures:
            lines.append("Nodes with most failures:")
            lines.append("")
            for node_name, count in failures[:5]:
                lines.append(f"- ❌ **{node_name}**: {count} failure(s)")
        else:
            lines.append("✅ No failures detected")
        
        lines.append("")
        
        # Slowest nodes
        lines.append("## Performance Bottlenecks")
        lines.append("")
        
        slowest = self.get_slowest_nodes(5)
        if slowest:
            lines.append("Slowest nodes (average duration):")
            lines.append("")
            for node_name, avg_duration in slowest:
                lines.append(f"- 🐌 **{node_name}**: {avg_duration:.2f}s")
        
        lines.append("")
        
        # Success rate trend
        lines.append("## Success Rate Trend")
        lines.append("")
        
        trend = self.get_success_rate_trend()
        if len(trend) >= 2:
            first = trend[0]
            last = trend[-1]
            change = last - first
            
            trend_icon = "📈" if change > 0 else "📉" if change < 0 else "➡️"
            lines.append(f"{trend_icon} Success rate: {first:.1f}% → {last:.1f}% ({change:+.1f}%)")
            lines.append("")
            lines.append("Recent runs:")
            for i, rate in enumerate(trend[-5:], 1):
                bars = "█" * int(rate / 10)
                lines.append(f"  Run {i}: {bars} {rate:.1f}%")
        
        lines.append("")
        
        # Recommendations
        lines.append("## Recommendations")
        lines.append("")
        
        if failures:
            lines.append(f"- 🔍 Investigate failures in **{failures[0][0]}** node")
        
        if slowest:
            lines.append(f"- ⚡ Optimize **{slowest[0][0]}** node (slowest on average)")
        
        if len(trend) >= 2 and trend[-1] < trend[0]:
            lines.append("- ⚠️ Success rate declining - review recent changes")
        
        return "\n".join(lines)

# Test with sample data
analyzer = StoryAnalyzer()

# Add sample stories
for i in range(5):
    sample_story = {
        "pipeline_name": "test",
        "started_at": f"2024-01-{15+i}T10:00:00",
        "duration": 10.0 + i * 0.5,
        "total_rows_processed": 5000 + i * 100,
        "success_rate": 100 - i * 5,
        "nodes": [
            {
                "node_name": "extract",
                "status": "success",
                "duration": 3.0 + i * 0.2
            },
            {
                "node_name": "transform",
                "status": "failed" if i % 2 == 1 else "success",
                "duration": 7.0 + i * 0.3
            }
        ]
    }
    analyzer.add_story(sample_story)

dashboard = analyzer.generate_dashboard()
print(dashboard)